In [216]:
import pandas as pd
import numpy as np
import seaborn as sns


In [230]:
houses_train = pd.read_csv('../data/processed_data.csv',keep_default_na=False)
#houses_test = pd.read_csv('../data/cleaned_test_data.csv',keep_default_na=False)

In [231]:
houses_train.head()

,MSSubClass,LotFrontage,LotArea,Street,Utilities,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,0.150685,0.033420,1.0,1.0,0.666667,0.500,0.949275,0.883333,0.12250,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,20,0.202055,0.038795,1.0,1.0,0.555556,0.875,0.753623,0.433333,0.00000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,60,0.160959,0.046507,1.0,1.0,0.666667,0.500,0.934783,0.866667,0.10125,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,70,0.133562,0.038561,1.0,1.0,0.666667,0.500,0.311594,0.333333,0.00000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,60,0.215753,0.060576,1.0,1.0,0.777778,0.500,0.927536,0.833333,0.21875,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [232]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [233]:
X = houses_train.drop(['SalePrice'],axis=1)
y = np.log10(houses_train['SalePrice'])

In [234]:
y

0       5.319106
1       5.258877
2       5.349278
3       5.146128
4       5.397940
          ...   
1455    5.243038
1456    5.322219
1457    5.425697
1458    5.152670
1459    5.168792
Name: SalePrice, Length: 1460, dtype: float64

In [245]:
X_train, X_val, y_train, y_val = train_test_split(X.copy(), y.copy(),
                                                                 shuffle = True,
                                                                 random_state = 321,
                                                                 test_size=.25)

In [246]:
model = RandomForestRegressor(n_estimators=100, random_state=0)

In [247]:
model.fit(X_train,y_train)

RandomForestRegressor(random_state=0)

In [248]:
A = model.predict(X_val)

In [249]:
mean_squared_error(y_val,A)

0.003751433787679264

In [250]:
import xgboost

xgb_reg = xgboost.XGBRegressor(n_estimators=200,
                          max_depth=10,
                          learning_rate=.1, enable_categorical=True)


xgb_reg.fit(X_train, y_train)

B = xgb_reg.predict(X_val)

In [251]:
mean_squared_error(y_val,B)

0.00443971197917987

In [253]:
from sklearn.linear_model import LinearRegression

lm = LinearRegression()

lm.fit(X_train,y_train)

D= lm.predict(X_val)

In [254]:
mean_squared_error(y_val,D)

0.0036695842840911617

In [267]:
lm.coef_

array([-1.99177746e-04,  7.91297020e-02,  2.79584118e-01,  4.69868722e-02,
        1.06434044e-01,  1.68269040e-01,  1.28457354e-01,  8.26114791e-02,
        1.34236457e-02,  1.54037076e-02, -5.01802744e-03, -1.58028947e-02,
        2.83224763e-02, -9.97266343e-03,  1.90118686e-02,  1.01679497e-02,
        1.81651069e-01,  2.60241152e-03,  1.90892410e-02,  2.01917920e-02,
        1.80121801e-01, -1.14209913e-02,  2.75139075e-02,  2.34693056e-02,
       -2.01472957e-02,  2.79517830e-01,  1.28988990e-01,  3.10324362e-02,
        2.83016489e-01,  4.09989293e-02,  2.24047047e-02,  1.66196752e-02,
        3.78858688e-02,  4.60453725e-03, -8.85339446e-02,  3.21107205e-02,
        2.93502886e-02,  9.40118200e-02,  4.15995397e-03,  6.57030382e-03,
        1.41253437e-02,  3.15208671e-03,  3.72851811e-02,  6.57586920e-02,
        4.69882152e-02,  2.35764620e-02,  6.47024265e-03,  3.34287084e-02,
        1.08588773e-02,  1.92067958e-02,  4.03882869e-02,  6.24320138e-02,
        8.89013830e-02, -

In [268]:
from sklearn.linear_model import Lasso 

lasso_lm = Lasso(alpha=1)

lasso_lm.fit(X_train,y_train)

E= lm.predict(X_val)

In [269]:
mean_squared_error(y_val,E)

0.0036695842840911617

In [270]:
lasso_lm.coef_

array([-0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,
        0., -0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,
        0.,  0., -0., -0.,  0., -0.,  0., -0.,  0., -0.,  0.,  0., -0.,
        0.,  0., -0.,  0.,  0., -0., -0.,  0.,  0., -0., -0., -0.,  0.,
        0.,  0., -0.,  0., -0., -0., -0., -0., -0.,  0.,  0.,  0., -0.,
       -0., -0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0.,  0., -0.,  0.,
        0.,  0., -0.,  0.,  0.,  0.,  0., -0., -0., -0., -0., -0.,  0.,
       -0., -0.,  0., -0.,  0., -0., -0., -0., -0.,  0.,  0.,  0., -0.,
        0.,  0., -0., -0.,  0.,  0., -0., -0.,  0., -0.,  0., -0.,  0.,
       -0.,  0.,  0., -0.,  0., -0., -0., -0., -0.,  0., -0.,  0., -0.,
        0., -0.,  0.,  0.,  0., -0.,  0., -0., -0.,  0., -0.,  0., -0.,
        0., -0., -0.,  0.,  0., -0.,  0., -0., -0., -0., -0., -0